In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from statistics import mode
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from collections import Counter

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_folder/univAI/Training Data.csv')

NameError: ignored

In [ ]:
train = df.copy()
train.head()
train = train.drop(['city','profession','state'],axis=1)
train = pd.get_dummies(train)

NameError: ignored

In [ ]:
risk_0 = train[train['risk_flag']==0]
risk_1 = train[train['risk_flag']==1]


NameError: ignored

In [ ]:
print(risk_0.shape,risk_1.shape)

(221004, 14) (30996, 14)


In [ ]:
train_y = train['risk_flag']
train_x = train.drop('risk_flag',axis=1) 

NameError: ignored

In [ ]:
x_train_res.shape,y_train_res.shape

((331506, 13), (331506,))

In [ ]:
x_train_res,x_test_res,y_train_res,y_test_res = train_test_split(x_train_res, y_train_res, test_size=0.33, random_state=42)
#random_classifier_model_categ2.fit(train_categ_2_x,train_categ_2_y)
#roc_auc_score(test_categ_2_y, random_classifier_model_categ2.predict_proba(test_categ_2_x)[:, 1])

In [ ]:
random_classifier = RandomForestClassifier()
random_classifier.fit(x_train_res,y_train_res)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
print(roc_auc_score(y_test_res, random_classifier.predict_proba(x_test_res)[:, 1]))
sol = random_classifier.predict_proba(x_test_res)[:, 1]
print(sol)
'''ans = []
for i in range(len(sol)):
    if sol[i] <= 0.5:
        ans.append(0)
    else:
        ans.append(1)
roc_auc_score(y_test_res, ans)'''

0.9901319742879063
[0.96 0.02 0.72 ... 0.   0.18 0.96]


'ans = []\nfor i in range(len(sol)):\n    if sol[i] <= 0.5:\n        ans.append(0)\n    else:\n        ans.append(1)\nroc_auc_score(y_test_res, ans)'

In [ ]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3,5,7,8,10,12,14, 15,20],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]    
}
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

classifier=xgboost.XGBClassifier()

random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)
from datetime import datetime
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(x_train_res,y_train_res)
timer(start_time)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: ignored

In [ ]:
random_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.0,
              learning_rate=0.1, max_delta_step=0, max_depth=14,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:

sol = classifier.predict_proba(x_test_res)[:, 1]
ans = []
for i in range(len(sol)):
    if sol[i] <= 0.5:
        ans.append(0)
    else:
        ans.append(1)
roc_auc_score(y_test_res, ans)

0.9268109045203001

In [ ]:
x_train_res

array([[ 148321, 5345253,      61, ...,       1,       1,       0],
       [ 181065, 5850867,      30, ...,       1,       0,       1],
       [ 248187, 1746935,      34, ...,       1,       1,       0],
       ...,
       [ 226713, 7255484,      40, ...,       1,       1,       0],
       [ 183422, 2299667,      24, ...,       1,       1,       0],
       [ 136407, 8026974,      23, ...,       1,       0,       1]])

In [ ]:
test.head()

,id,income,age,experience,current_job_years,current_house_years,married_married,married_single,house_ownership_norent_noown,house_ownership_owned,house_ownership_rented,car_ownership_no,car_ownership_yes
0,1,7393090,59,19,4,13,0,1,0,0,1,1,0
1,2,1215004,25,5,5,10,0,1,0,0,1,1,0
2,3,8901342,50,12,9,14,0,1,0,0,1,1,0
3,4,1944421,49,9,3,12,1,0,0,0,1,0,1
4,5,13429,25,18,13,11,0,1,0,0,1,0,1


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_folder/univAI/Test Data.csv')
test = test.drop(['city','profession','state'],axis = 1)
test = pd.get_dummies(test)

test = test.to_numpy()
sol = random_classifier.predict_proba(test)
ans = []
print(sol[:,1])

for i in range(len(sol)):
    ans.append([i+1,sol[i][1]])

df = pd.DataFrame(ans, columns = ['id', 'risk_flag'])
df.to_csv('sub_file.csv')

[0.04 0.02 0.09 ... 0.09 0.04 0.28]


#Ensemble tech

In [ ]:

model2 = DecisionTreeClassifier()
model3 = KNeighborsClassifier()
model4 = RandomForestClassifier()

In [ ]:
#model1.fit(x_train_res,y_train_res)
model2.fit(x_train_res,y_train_res)
model3.fit(x_train_res,y_train_res)
model4.fit(x_train_res,y_train_res)

#pred1=model1.predict(x_test_res)
pred2=model2.predict(x_test_res)
pred3=model3.predict(x_test_res)
pred4=model4.predict(x_test_res)

#print(pred1.shape)
print(pred2.shape)
print(pred3.shape)
print(pred4.shape)

final_pred = np.array([])
for i in range(0,len(x_test_res)):
    #print([pred1[i], pred2[i], pred3[i],pred4[i]])
    ct = Counter([ pred2[i], pred3[i],pred4[i]])
    final_pred = np.append(final_pred,max(ct, key=ct.get))
    #final_pred = np.append(final_pred, mode([pred1[i], pred2[i], pred3[i],pred4[i]]))



roc_auc_score(y_test_res, final_pred)


(109397,)
(109397,)
(109397,)


IndexError: ignored

In [ ]:
final_pred = np.array([])
for i in range(0,len(x_test_res)):
    #print([pred1[i], pred2[i], pred3[i],pred4[i]])
    ct = mode([pred1[i], pred2[i], pred3[i],pred4[i]])
    final_pred = np.append(final_pred,ct)
    #final_pred = np.append(final_pred, mode([pred1[i], pred2[i], pred3[i],pred4[i]]))


roc_auc_score(y_test_res, final_pred)


StatisticsError: ignored

In [ ]:

test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_folder/univAI/Test Data.csv')
test = test.drop(['city','profession','state'],axis = 1)
test = pd.get_dummies(test)

test = test.to_numpy()
#pred1=model1.predict(test)
pred2=model2.predict(test)
pred3=model3.predict(test)
pred4=model4.predict(test)


final_pred = np.array([])
for i in range(0,len(test)):
    #print([pred1[i], pred2[i], pred3[i],pred4[i]])
    ct = Counter([ pred2[i], pred3[i],pred4[i]])
    final_pred = np.append(final_pred,max(ct, key=ct.get))
    #final_pred = np.append(final_pred, mode([pred1[i], pred2[i], pred3[i],pred4[i]]))



sol = final_pred
ans = []
print(sol)
for i in range(len(sol)):
    ans.append([i+1,sol[i]])
    
df = pd.DataFrame(ans, columns = ['id', 'risk_flag'])
df.to_csv('sub_file.csv')



'''
final_pred = np.array([])
for i in range(0,len(test)):
    #print([pred1[i], pred2[i], pred3[i],pred4[i]])
    ct = Counter([pred1[i], pred2[i], pred3[i],pred4[i]])
    final_pred = np.append(final_pred,max(ct, key=ct.get))
    #final_pred = np.append(final_pred, mode([pred1[i], pred2[i], pred3[i],pred4[i]]))



roc_auc_score(y_test_res, final_pred)
'''

[0. 0. 0. ... 1. 0. 0.]


'\nfinal_pred = np.array([])\nfor i in range(0,len(test)):\n    #print([pred1[i], pred2[i], pred3[i],pred4[i]])\n    ct = Counter([pred1[i], pred2[i], pred3[i],pred4[i]])\n    final_pred = np.append(final_pred,max(ct, key=ct.get))\n    #final_pred = np.append(final_pred, mode([pred1[i], pred2[i], pred3[i],pred4[i]]))\n\n\n\nroc_auc_score(y_test_res, final_pred)\n'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_folder/univAI/Training Data.csv')

In [ ]:
import category_encoders as ce

In [ ]:
x_train_res,x_test_res,y_train_res,y_test_res 

(array([[ 146364, 4143099,      21, ...,       0,       0,       0],
        [  66672, 2415720,      79, ...,       0,       0,       0],
        [  30072, 6000177,      70, ...,       0,       0,       0],
        ...,
        [ 131933, 9013930,      25, ...,       0,       0,       0],
        [ 146868, 6660597,      40, ...,       0,       0,       0],
        [ 121959, 2984988,      70, ...,       0,       0,       0]]),
 array([[ 157552, 7794515,      51, ...,       0,       0,       0],
        [ 114257, 4157390,      34, ...,       0,       0,       1],
        [   5225, 9633948,      78, ...,       0,       0,       0],
        ...,
        [ 170835, 7350053,      75, ...,       0,       0,       0],
        [ 163208, 1514602,      71, ...,       0,       0,       0],
        [   8889, 3413189,      60, ...,       0,       0,       0]]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([1, 0, 1, ..., 0, 0, 1]))

In [ ]:
random_classifier = RandomForestClassifier()
random_classifier.fit(x_train_res,y_train_res)


KeyboardInterrupt: ignored

In [ ]:
print(roc_auc_score(y_test_res, random_classifier.predict_proba(x_test_res)[:, 1]))
sol = random_classifier.predict_proba(x_test_res)[:, 1]
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_folder/univAI/Test Data.csv')
test = test.drop(['city','profession','state'],axis = 1)
test = pd.get_dummies(test)

test = test.to_numpy()
sol = random_classifier.predict_proba(test)
ans = []
print(sol)

for i in range(len(sol)):
    if sol[i][1] < 0.5:
        ans.append([i+1,0])
    else:
        ans.append([i+1,1])
        
df = pd.DataFrame(ans, columns = ['id', 'risk_flag'])
df.to_csv('sub_file.csv')

In [ ]:
import re

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost




'''xgb_train_x = x_train_res.copy()
xgb_train_x.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in xgb_train_x.columns.values]
'''# fit xgmodel no training data

#xgbmodel = xgboost.XGBClassifier(n_estimators=100)
#xgbmodel.fit(xgb_train_x,train_categ_2_y)

In [ ]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [  15,20,22,24,27,30,25,42],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}
classifier=xgboost.XGBClassifier()

random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

random_search.fit(x_train_res,y_train_res)



Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
exception calling callback for <Future at 0x7f1f35b474d0 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 792, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/usr/local/lib/python3.7/dist-packages/joblib/_parallel_backends.py", line 531, in apply_async
 

TerminatedWorkerError: ignored

factorize

In [ ]:
pip install category_encoders

     |████████████████████████████████| 81kB 4.2MB/s 


In [ ]:
train = df.copy()
train.head()

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1,1303835,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


In [ ]:
import category_encoders as ce
encoder_city=ce.TargetEncoder(cols='city') 
encoder_prof=ce.TargetEncoder(cols='profession') 
encoder_state=ce.TargetEncoder(cols='state') 

In [ ]:
train['city'] = encoder_city.fit_transform(train['city'],train['risk_flag'])
train['profession'] = encoder_prof.fit_transform(train['profession'],train['risk_flag'])
train['state'] = encoder_state.fit_transform(train['state'],train['risk_flag'])

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
train = pd.get_dummies(train)
train

,Id,income,age,experience,profession,city,state,current_job_years,current_house_years,risk_flag,married_married,married_single,house_ownership_norent_noown,house_ownership_owned,house_ownership_rented,car_ownership_no,car_ownership_yes
0,1,1303835,23,3,0.111558,0.121554,0.154369,3,13,0,0,1,0,0,1,1,0
1,2,7574516,40,10,0.148427,0.122497,0.113254,9,13,0,0,1,0,0,1,1,0
2,3,3991815,66,4,0.134167,0.130814,0.167097,4,10,0,1,0,0,0,1,1,0
3,4,6256451,41,2,0.148427,0.326194,0.142550,2,12,1,0,1,0,0,1,0,1
4,5,5768871,47,11,0.115794,0.121137,0.103163,3,14,1,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251995,251996,8154883,43,13,0.115465,0.081317,0.128135,6,11,0,0,1,0,0,1,1,0
251996,251997,2843572,26,10,0.152113,0.121554,0.154369,6,11,0,0,1,0,0,1,1,0
251997,251998,4522448,46,7,0.106999,0.086370,0.113254,7,12,0,0,1,0,0,1,1,0
251998,251999,6507128,45,0,0.115370,0.089186,0.116539,0,10,0,0,1,0,0,1,1,0


In [ ]:
train_y = train['risk_flag']
train_x = train.drop('risk_flag',axis=1) 
os =  RandomOverSampler(ratio=0.5)
x_train_res, y_train_res = os.fit_sample(train_x, train_y)
x_train_res,x_test_res,y_train_res,y_test_res = train_test_split(x_train_res, y_train_res, test_size=0.33, random_state=42)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
model2 = DecisionTreeClassifier()
model3 = KNeighborsClassifier()
model4 = RandomForestClassifier()
#model1.fit(x_train_res,y_train_res)
model2.fit(x_train_res,y_train_res)
model3.fit(x_train_res,y_train_res)
model4.fit(x_train_res,y_train_res)

#pred1=model1.predict(x_test_res)
pred2=model2.predict(x_test_res)
pred3=model3.predict(x_test_res)
pred4=model4.predict(x_test_res)

#print(pred1.shape)
print(pred2.shape)
print(pred3.shape)
print(pred4.shape)

final_pred = np.array([])
for i in range(0,len(x_test_res)):
    #print([pred1[i], pred2[i], pred3[i],pred4[i]])
    ct = Counter([ pred2[i], pred3[i],pred4[i]])
    final_pred = np.append(final_pred,max(ct, key=ct.get))
    #final_pred = np.append(final_pred, mode([pred1[i], pred2[i], pred3[i],pred4[i]]))
roc_auc_score(y_test_res, final_pred)


(109397,)
(109397,)
(109397,)


0.9575685911146475

In [ ]:

test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_folder/univAI/Test Data.csv')

test['city'] = encoder_city.fit_transform(test['city'],test['risk_flag'])
test['profession'] = encoder_prof.fit_transform(test['profession'],test['risk_flag'])
test['state'] = encoder_state.fit_transform(test['state'],test['risk_flag'])


test = pd.get_dummies(test)

test = test.to_numpy()
#pred1=model1.predict(test)
pred2=model2.predict(test)
pred3=model3.predict(test)
pred4=model4.predict(test)


final_pred = np.array([])
for i in range(0,len(test)):
    #print([pred1[i], pred2[i], pred3[i],pred4[i]])
    ct = Counter([ pred2[i], pred3[i],pred4[i]])
    final_pred = np.append(final_pred,max(ct, key=ct.get))
    #final_pred = np.append(final_pred, mode([pred1[i], pred2[i], pred3[i],pred4[i]]))



sol = final_pred
ans = []
print(sol)
for i in range(len(sol)):
    ans.append([i+1,sol[i]])
    
df = pd.DataFrame(ans, columns = ['id', 'risk_flag'])
df.to_csv('sub_file.csv')



'''
final_pred = np.array([])
for i in range(0,len(test)):
    #print([pred1[i], pred2[i], pred3[i],pred4[i]])
    ct = Counter([pred1[i], pred2[i], pred3[i],pred4[i]])
    final_pred = np.append(final_pred,max(ct, key=ct.get))
    #final_pred = np.append(final_pred, mode([pred1[i], pred2[i], pred3[i],pred4[i]]))



roc_auc_score(y_test_res, final_pred)
'''

KeyError: ignored